# Extracción de categorías del libro.

In [ ]:
import requests
from bs4 import BeautifulSoup

# Defino la base del sitio principal
BASE_URL = "https://books.toscrape.com/"

# Creo la URL inicial del sitio y uso BASE_URL + url_relativa para armar URLs completas de las categorías
CATEGORIAS_URL = BASE_URL + "index.html" 

# Extraigo con el método get las categorias y con el "parser" obtengo la info relevante
respuesta = requests.get(CATEGORIAS_URL)
soup = BeautifulSoup(respuesta.text, "html.parser")

# Extaigo los enlaces de categorias (hay uno para cada categoria dentro de 'ul.nav-list')
categorias_links = soup.select("ul.nav-list ul li a")

categoria = {} # Creo un diccionario vacio para guardar categorias

for link in categorias_links: 
    nombre = link.text.strip() # Obtengo y limpio el nombre de la categoria
    url_relativa = link['href'] # Saco la URL relativa del href
    url_completa = BASE_URL + url_relativa # Armo la URL completa
    categoria[nombre] = url_completa # Guardo en el diccionario: nombre -> URL

print(f"Categorias encontradas: {len(categoria)}\n")

# Recorre cada par del diccionario : (clave,valor) y con el .items devuelve la lista de pares
for nombre, url in categoria.items():
    print(f"- {nombre}: {url}")
    




# Obtención de libros por categoría


In [20]:
def obtener_autor(url_libro):
    response = requests.get(url_libro)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # El autor está en la segunda fila de la tabla (después del UPC)
        info_table = soup.select('table.table.table-striped tr')
        if len(info_table) > 1:
            autor = info_table[1].text.strip()
            print(f"Autor encontrado: {autor}")
            return autor
        else:
            print(f"No se encontró autor en: {url_libro}")
            return "Autor no encontrado"
    else:
        print(f"Error al acceder a: {url_libro}")
        return "Error"


In [2]:
import time # Para no hacer pedidos(requests) una tras otra, porque puede causar problemas

"""Creo una función y le doy dos argumentos"""
def obtener_libros_por_categoria(nombre_categoria,url_categoria):

    
    pagina_actual = url_categoria #Asigna la URL inicial de la categoría a una variable que iremos actualizando
    libros = [] # Lista vacía en donde se va guardar un diccionario por cada libro

    while True:
        print(f"Scrapeando {pagina_actual}...")
        respuesta = requests.get(pagina_actual)
        soup = BeautifulSoup(respuesta.text, "html.parser")

        libros_en_pagina = soup.select(".product_pod")
        
        for libro in libros_en_pagina:
            titulo = libro.h3.a["title"]
            precio = libro.select_one(".price_color").text.strip()
            rating = libro.p["class"][1]

            url_relativa = libro.h3.a["href"]
            url_libro = "/".join(pagina_actual.split("/")[:-1]) + "/" + url_relativa

            

            libros.append({
                "categoria": nombre_categoria,
                "titulo": titulo,
                "precio": precio,
                "rating": rating,
                "url_libro": url_libro,
               
            })


        next_button = soup.select_one("li.next a") # Busca si hay un enlace a la próxima página. Si existe, sigue
        # Construye la URL de la próxima página
        if next_button:
            next_url = next_button["href"]
            # Si pagina_actual = ".../index.html" y next_url = "page-2.html"➡ Reemplaza index.html con page-2.html
            if "index.html" in pagina_actual:
                pagina_actual = pagina_actual.replace("index.html", next_url )
            else:
                # Esta línea se ejecuta cuando hay más de una página en la categoría (ejemplo: page-2.html, page-3.html, etc.).
                pagina_actual = "/".join(pagina_actual.split("/")[:-1])+ "/" + next_url
        else: 
            break

        time.sleep(1) # Espera 1 segundo antes de pasar a la próxima página
    return libros # Devuelve la lista de diccionarios con todos los libros encontrados

# Visualización de los resultados

In [3]:
#  lista vacía para juntar todos los libros del sitio
todos_los_libros = []
# Recorre todas las categorías que ya tenemos en el diccionario
for nombre,url in categoria.items():
    print(f"Scrapeando categoría: {nombre}...")

    libros_categoria = obtener_libros_por_categoria(nombre,url)

    todos_los_libros.extend(libros_categoria)  # agregamos todos los libros a la lista general
print(f"\nScraping completado. Total de libros encontrados: {len(todos_los_libros)}")

Scrapeando categoría: Travel...
Scrapeando https://books.toscrape.com/catalogue/category/books/travel_2/index.html...
Scrapeando categoría: Mystery...
Scrapeando https://books.toscrape.com/catalogue/category/books/mystery_3/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html...
Scrapeando categoría: Historical Fiction...
Scrapeando https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html...
Scrapeando categoría: Sequential Art...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-2.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-3.html...
Scrapeando https://books.toscrape.com/catalogue/category/books/sequential-art_5/page-4.html...
Scrapeando cat

 # Creación de Base de datos en SQLite usando los datos ya scrapeados. 

In [1]:
import sqlite3

# Crear conexión y cursor
conn = sqlite3.connect("libros.db") # Crea el archivo libros.db
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS libros(
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      titulo TEXT,
      categoria TEXT,
      precio REAL,
      rating TEXT                       
)
""")
conn.commit()

# Inserción de libros (desde todos_los_libros)

In [11]:
import re # Expresiones regulares

for libro in todos_los_libros:
    titulo = libro["titulo"]
    categoria = libro["categoria"]

    # "sub" → sustituye , "r"[^\d.]" → todo lo que no sea número ni punto ,"" → lo reemplaza por vacío (lo elimina)
    precio_str = re.sub(r"[^\d.]", "", libro["precio"])
    precio = float(precio_str)

    rating = libro["rating"]

    cursor.execute("""
        INSERT INTO libros(titulo, categoria,precio,rating)
        VALUES(?,?,?,?)
    """,(titulo,categoria,precio,rating))
conn.commit() # guardar los cambios en la base de datos

 # Ver si hay libros con precio cero (por error)

In [12]:
cursor.execute("SELECT COUNT(*) FROM libros where precio = 0") #COUNT(*) → Cuenta cuántas filas cumplen la condición.
print(f"Libros con precio 0: {cursor.fetchone()[0]}") # fetchone() devuelve una tupla

Libros con precio 0: 0


 # Obtener precio promedio

In [ ]:
"""Calcula el promedio (average) de todos los precios en la tabla.
  La función fetchone()[0] obtiene ese valor único.
  El formato :.2f lo muestra con 2 decimales, estilo monetario."""
cursor.execute("SELECT AVG(precio) FROM libros")
print(f"Precio promedio : {cursor.fetchone()[0]:.2f}")

Precio promedio : 35.07


# Ver algunos libros con sus precios y ratings

In [ ]:
cursor.execute("SELECT titulo,precio,rating FROM libros LIMIT 5")
for fila in cursor.fetchall():
    print(fila)
